In [78]:
import pandas as pd
import numpy as np
import geopandas as gpd
%matplotlib inline

In [2]:
df = pd.read_csv('/home/data/spring_2019/base/sfbay-smart-base__2019-03-28_14-22-12/ITERS/it.30/30.skims.csv')

In [3]:
df.head()

,hour,mode,origTaz,destTaz,travelTimeInS,generalizedTimeInS,cost,generalizedCost,distanceInM,numObservations
0,16,CAR,975,31,5781.0,5561.027549,0.000000,15.901631,87041.544,1
1,5,CAR,262,194,1118.0,1118.000000,2.535153,21.031476,23114.210,1
2,18,CAR,33,175,1956.0,1943.730664,1.121041,12.634152,10221.073,1
3,8,CAR,711,710,576.0,576.000000,0.000000,3.921569,6977.472,1
4,7,CAR,731,1106,1657.0,1657.000000,0.000000,10.153186,44510.120,1


In [21]:
mtc = pd.read_csv('/home/data/fall_2018/mtc_skims/DistanceSkimsDatabaseAM.csv', usecols=['orig', 'dest', 'da'], index_col=['orig', 'dest'])

/home/max/anaconda3/envs/ual-model/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [59]:
mtc['dist'] = mtc['da'] * 1609.34

In [28]:
df['gen_time_per_m'] = df['generalizedTimeInS'] / df['distanceInM'] 
df['gen_cost_per_m'] = df['generalizedCost'] / df['distanceInM']
df.rename(columns={'origTaz': 'orig', 'destTaz': 'dest'}, inplace=True)
df.loc[df['hour'].isin([7,8,9]), 'period'] = 'AM'
# df.loc[df['hour'].isin([17,18,19]), 'period'] = 'PM'
df_am = df[df['period'] == 'AM']

In [29]:
df_am = df_am.replace([np.inf, -np.inf], np.nan)
am_lookup = df_am[['mode', 'gen_time_per_m', 'gen_cost_per_m']].dropna().groupby(['mode']).mean().reset_index()

In [30]:
am_lookup

,mode,gen_time_per_m,gen_cost_per_m
0,BIKE,0.250118,0.001220
1,CAR,0.065745,0.000689
2,DRIVE_TRANSIT,0.109953,0.000914
3,RIDE_HAIL,0.217711,0.004927
4,RIDE_HAIL_POOLED,0.102425,0.001940
5,RIDE_HAIL_TRANSIT,0.098926,0.001636
6,WALK,1.092729,0.349632
7,WALK_TRANSIT,0.165653,0.001095


In [31]:
df_am_avg = df_am[['orig', 'dest', 'mode', 'generalizedTimeInS', 'generalizedCost']].groupby(
    ['orig', 'dest', 'mode']).mean().reset_index()

In [32]:
df_am_avg.head()

,orig,dest,mode,generalizedTimeInS,generalizedCost
0,1,10,WALK,2369.0,4.919288
1,1,31,RIDE_HAIL,363.0,6.181276
2,1,37,WALK_TRANSIT,314.0,7.795207
3,1,44,WALK_TRANSIT,2020.0,5.775599
4,1,49,WALK_TRANSIT,2175.0,6.066993


In [34]:
df_am_pivot = df_am_avg.pivot_table(index=['orig', 'dest'], columns='mode')

In [35]:
df_am_pivot.columns = ['_'.join(col) for col in df_am_pivot.columns.values]

In [36]:
df_am_pivot.head()

generalizedCost_BIKE  generalizedCost_CAR  \
orig dest                                              
1    10                     NaN                  NaN   
     31                     NaN                  NaN   
     37                     NaN                  NaN   
     44                     NaN                  NaN   
     49                     NaN                  NaN   

           generalizedCost_DRIVE_TRANSIT  generalizedCost_RIDE_HAIL  \
orig dest                                                             
1    10                              NaN                        NaN   
     31                              NaN                   6.181276   
     37                              NaN                        NaN   
     44                              NaN                        NaN   
     49                              NaN                        NaN   

           generalizedCost_RIDE_HAIL_POOLED  \
orig dest                                     
1    10                                 NaN   
     31                                 NaN   
     37                                 NaN   
     44                                 NaN   
     49                                 NaN   

           generalizedCost_RIDE_HAIL_TRANSIT  generalizedCost_WALK  \
orig dest                                                            
1    10                                  NaN              4.919288   
     31                                  NaN                   NaN   
     37                                  NaN                   NaN   
     44                                  NaN                   NaN   
     49                                  NaN                   NaN   

           generalizedCost_WALK_TRANSIT  generalizedTimeInS_BIKE  \
orig dest                                                          
1    10                             NaN                      NaN   
     31                             NaN                      NaN   
     37                        7.795207                      NaN   
     44                        5.775599                      NaN   
     49                        6.066993                      NaN   

           generalizedTimeInS_CAR  generalizedTimeInS_DRIVE_TRANSIT  \
orig dest                                                             
1    10                       NaN                               NaN   
     31                       NaN                               NaN   
     37                       NaN                               NaN   
     44                       NaN                               NaN   
     49                       NaN                               NaN   

           generalizedTimeInS_RIDE_HAIL  generalizedTimeInS_RIDE_HAIL_POOLED  \
orig dest                                                                      
1    10                             NaN                                  NaN   
     31                           363.0                                  NaN   
     37                             NaN                                  NaN   
     44                             NaN                                  NaN   
     49                             NaN                                  NaN   

           generalizedTimeInS_RIDE_HAIL_TRANSIT  generalizedTimeInS_WALK  \
orig dest                                                                  
1    10                                     NaN                   2369.0   
     31                                     NaN                      NaN   
     37                                     NaN                      NaN   
     44                                     NaN                      NaN   
     49                                     NaN                      NaN   

           generalizedTimeInS_WALK_TRANSIT  
orig dest                                   
1    10                                NaN  
     31                                NaN  
     37                              314.0  
     

In [60]:
merged = pd.merge(mtc[['dist']], df_am_pivot, left_index=True, right_index=True, how='left')

In [61]:
merged.head()

dist  generalizedCost_BIKE  generalizedCost_CAR  \
orig dest                                                         
1    1      193.1208                   NaN                  NaN   
     2      386.2416                   NaN                  NaN   
     3      708.1096                   NaN                  NaN   
     4      659.8294                   NaN                  NaN   
     5     1094.3512                   NaN                  NaN   

           generalizedCost_DRIVE_TRANSIT  generalizedCost_RIDE_HAIL  \
orig dest                                                             
1    1                               NaN                        NaN   
     2                               NaN                        NaN   
     3                               NaN                        NaN   
     4                               NaN                        NaN   
     5                               NaN                        NaN   

           generalizedCost_RIDE_HAIL_POOLED  \
orig dest                                     
1    1                                  NaN   
     2                                  NaN   
     3                                  NaN   
     4                                  NaN   
     5                                  NaN   

           generalizedCost_RIDE_HAIL_TRANSIT  generalizedCost_WALK  \
orig dest                                                            
1    1                                   NaN                   NaN   
     2                                   NaN                   NaN   
     3                                   NaN                   NaN   
     4                                   NaN                   NaN   
     5                                   NaN                   NaN   

           generalizedCost_WALK_TRANSIT  generalizedTimeInS_BIKE  \
orig dest                                                          
1    1                              NaN                      NaN   
     2                              NaN                      NaN   
     3                              NaN                      NaN   
     4                              NaN                      NaN   
     5                              NaN                      NaN   

           generalizedTimeInS_CAR  generalizedTimeInS_DRIVE_TRANSIT  \
orig dest                                                             
1    1                        NaN                               NaN   
     2                        NaN                               NaN   
     3                        NaN                               NaN   
     4                        NaN                               NaN   
     5                        NaN                               NaN   

           generalizedTimeInS_RIDE_HAIL  generalizedTimeInS_RIDE_HAIL_POOLED  \
orig dest                                                                      
1    1                              NaN                                  NaN   
     2                              NaN                                  NaN   
     3                              NaN                                  NaN   
     4                              NaN                                  NaN   
     5                              NaN                                  NaN   

           generalizedTimeInS_RIDE_HAIL_TRANSIT  generalizedTimeInS_WALK  \
orig dest                                                                  
1    1                                      NaN                      NaN   
     2                                      NaN                      NaN   
     3                                      NaN                      NaN   
     4                                      NaN                      NaN   
     5                                      NaN                      NaN   

           generalizedTimeInS_WALK_TRANSIT  
orig dest                                   
1    1                                 NaN  
     2                 

In [63]:
for mode in am_lookup['mode'].values:
    for impedance in ['generalizedTimeInS', 'generalizedCost']:
        if impedance == 'generalizedTimeInS':
            lookup_col = 'gen_time_per_m'
        elif impedance == 'generalizedCost':
            lookup_col = 'gen_cost_per_m'
        colname = impedance + '_' + mode
        merged.loc[pd.isnull(merged[colname]), colname] = merged.loc[
            pd.isnull(merged[colname]), 'dist'] * am_lookup.loc[am_lookup['mode'] == mode, lookup_col].values[0]

In [66]:
merged.reset_index(inplace=True)

In [69]:
excerpts = pd.read_csv('/home/data/spring_2019/base/sfbay-smart-base__2019-03-28_14-22-12/ITERS/it.30/30.skimsExcerpt.csv')

In [75]:
excerpts[excerpts['period'] == 'AM'].sort_values(['origTaz', 'destTaz']).head(10)

,period,mode,origTaz,destTaz,travelTimeInS,generalizedTimeInS,cost,generalizedCost,distanceInM,numObservations
1069425,AM,CAR,1,1,15.0,15.0,0.015794,0.049127,144.0,2.0
1069428,AM,CAR,1,2,0.0,0.0,0.000000,0.000000,0.0,2.0
1069383,AM,CAR,1,3,90.0,90.0,0.079299,0.279299,723.0,2.0
1069389,AM,CAR,1,4,70.0,70.0,0.057034,0.212589,520.0,2.0
1069377,AM,CAR,1,5,132.0,132.0,0.106499,0.399833,971.0,2.0
1069344,AM,CAR,1,6,203.0,203.0,0.165068,0.616180,1505.0,2.0
1069374,AM,CAR,1,7,180.0,180.0,0.140281,0.540281,1279.0,2.0
1069347,AM,CAR,1,8,203.0,203.0,0.164520,0.615631,1500.0,2.0
1069293,AM,CAR,1,9,383.0,383.0,0.303704,1.154815,2769.0,2.0
1069296,AM,CAR,1,10,386.0,386.0,0.307433,1.165211,2803.0,2.0


In [71]:
merged[['orig', 'dest', 'dist', 'generalizedCost_CAR', 'generalizedTimeInS_CAR']]

,orig,dest,dist,generalizedCost_CAR,generalizedTimeInS_CAR
0,1,1,193.1208,0.133005,12.696789
1,1,2,386.2416,0.266009,25.393578
2,1,3,708.1096,0.487684,46.554893
3,1,4,659.8294,0.454433,43.380696
4,1,5,1094.3512,0.753693,71.948471
5,1,6,1561.0598,1.075121,102.632377
6,1,7,1577.1532,1.086205,103.690443
7,1,8,1850.7410,1.274628,121.677561
8,1,9,2526.6638,1.740145,166.116322
9,1,10,2558.8506,1.762312,168.232454


### Validation

In [76]:
zones_w_aggs = pd.read_csv('/home/data/spring_2019/base/zones_w_access_vars.csv')

In [77]:
zones_w_aggs.head()

,zone_id,gid,area,acres,total_jobs,sum_residential_units,sum_persons,sum_income,total_jobs_gen_tt_CAR_15,total_jobs_gen_tt_CAR_45,...,sum_residential_units_gen_tt_WALK_TRANSIT_15,sum_residential_units_gen_tt_WALK_TRANSIT_45,total_jobs_gen_tt_RIDE_HAIL_15,total_jobs_gen_tt_RIDE_HAIL_45,sum_persons_gen_tt_RIDE_HAIL_15,sum_persons_gen_tt_RIDE_HAIL_45,sum_income_gen_tt_RIDE_HAIL_15,sum_income_gen_tt_RIDE_HAIL_45,sum_residential_units_gen_tt_RIDE_HAIL_15,sum_residential_units_gen_tt_RIDE_HAIL_45
0,1442,525,5272042.0,121.028999,2194.0,2567.0,5839.0,477004000.0,36883.0,215824.0,...,16368.0,64675.0,9721.0,43462.0,24549.0,96415.0,1.945550e+09,6.928568e+09,11508.0,47434.0
1,1434,508,7820341.5,179.529999,771.0,2425.0,5237.0,313941043.0,16326.0,97976.0,...,12751.0,57822.0,4157.0,44280.0,20950.0,92538.0,1.249575e+09,5.414574e+09,9877.0,43116.0
2,246,784,4247867.5,97.517998,4260.0,2946.0,7469.0,511362913.0,40830.0,240331.0,...,23321.0,139485.0,24519.0,90401.0,31700.0,184867.0,2.007720e+09,9.497006e+09,12770.0,76596.0
3,1312,560,759732480.0,17441.058594,220.0,1018.0,1591.0,65422447.0,220.0,440.0,...,1018.0,2036.0,220.0,440.0,1591.0,3182.0,6.542245e+07,1.308449e+08,1018.0,2036.0
4,705,1205,27835386.0,639.013000,948.0,3682.0,9915.0,372431913.0,9024.0,29283.0,...,15952.0,26407.0,3300.0,17637.0,34907.0,65915.0,1.182720e+09,2.245544e+09,12136.0,22453.0


In [79]:
gdf = gpd.read_file('/home/data/spring_2019/base/Transportation_Analysis_Zones.shp')

In [82]:
gdf = gdf.merge(
    zones_w_aggs[['total_jobs_gen_tt_CAR_15', 'total_jobs_gen_tt_CAR_45', 'zone_id']],
    left_on='taz1454', right_on='zone_id')

In [83]:
gdf.plot()

1454